# Flower classifier using  pytorch CNN

### install pytorch

In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision


    100% |████████████████████████████████| 484.0MB 66.4MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
fastai 1.0.45 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
    100% |████████████████████████████████| 2.0MB 13.8MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.45 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


### Download train, test and validation dataset

In [2]:
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
#!wget -cq https://github.com/cahya-wirawan/Udacity-Course/blob/master/flower_test.tgz
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!rm -r flower_data || true
!unzip -qq flower_data.zip
!pip install gdown==3.6.0
my_file_id = "1fMJuaUx3V8x4Xqz30Zkj8xNELs2Or2eH"
!gdown https://drive.google.com/uc?id={my_file_id}
!unzip -qq flower_test.zip

rm: cannot remove 'flower_data': No such file or directory
  Stored in directory: /root/.cache/pip/wheels/97/90/fa/25654eb65da3e6da7752db71a164e0eb8f7a6fb4335eeb46ab
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4
Downloading...
From: https://drive.google.com/uc?id=1fMJuaUx3V8x4Xqz30Zkj8xNELs2Or2eH
To: /content/flower_test.zip
35.1MB [00:00, 56.1MB/s]


In [0]:
data_dir = '/content/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = "/content/flower_test/test"


### Import dependencies and define transforms

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os



train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])


validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])




### Load dataset and define dataloaders

In [0]:
# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_data = datasets.ImageFolder(valid_dir, transform=validation_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
vloader = torch.utils.data.DataLoader(validation_data, batch_size =32,shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size =32,shuffle = True)


## Mount google drive to access files

In [6]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##load pretrained model

In [7]:
model = models.resnet152(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
print(model)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:02<00:00, 104674402.05it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [8]:
#path = F"/content/drive/My Drive/checkpoint.pt"

#model.eval()
#for param in model.parameters():
#    param.requires_grad = True
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 512)),
                          ('relu', nn.ReLU()),
                         # ( 'dropout',nn.Dropout(0.2)),
                          ('fc2', nn.Linear(512, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.fc = classifier

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = 0.003, )
#scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.3)
#model.load_state_dict(torch.load("/content/chk.pt"))

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [9]:

epochs = 30
print_every = 5
steps = 0
loss_show=[]
accuracy_min = 0
# change to cuda
model.to('cuda')
#path = F"/content/drive/My Drive/checkpoint.pt"
for e in range(epochs):
    running_loss = 0
    
    for ii, (inputs, labels) in enumerate(trainloader):
        steps += 1
        
        inputs,labels = inputs.to('cuda'), labels.to('cuda')
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            vlost = 0
            accuracy=0
            
            
            for ii, (inputs2,labels2) in enumerate(vloader):
                optimizer.zero_grad()
                
                inputs2, labels2 = inputs2.to('cuda:0') , labels2.to('cuda:0')
                model.to('cuda:0')
                with torch.no_grad():    
                    outputs = model.forward(inputs2)
                    vlost = criterion(outputs,labels2)
                    ps = torch.exp(outputs).data
                    equality = (labels2.data == ps.max(1)[1])
                    accuracy += equality.type_as(torch.FloatTensor()).mean()
                    
            vlost = vlost / len(vloader)
            accuracy = accuracy /len(vloader)
            
                    
            
            print("Epoch: {}/{}... ".format(e+1, epochs),
                  "Loss: {:.4f}".format(running_loss/print_every),
                  "Validation Lost {:.4f}".format(vlost),
                   "Accuracy: {:.4f}".format(accuracy))
            
            
             # save model if validation loss has decreased
            if accuracy> accuracy_min:
                  print('accuracy increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                  accuracy_min,
                     accuracy))
                  
                  
                  #chpt = torch.load(path)
                  #model.load_state_dict(chpt)
                  #model.class_to_idx = train_data.class_to_idx
                  
                  torch.save(model.state_dict(),'chk.pt')
                  
                  
                  accuracy_min = accuracy
            running_loss = 0
            #model.train()


Epoch: 1/30...  Loss: 5.1406 Validation Lost 0.1913 Accuracy: 0.0192
accuracy increased (0.000000 --> 0.019231).  Saving model ...
Epoch: 1/30...  Loss: 4.5916 Validation Lost 0.1703 Accuracy: 0.0731
accuracy increased (0.019231 --> 0.073050).  Saving model ...
Epoch: 1/30...  Loss: 4.2944 Validation Lost 0.1635 Accuracy: 0.1031
accuracy increased (0.073050 --> 0.103098).  Saving model ...
Epoch: 1/30...  Loss: 4.0557 Validation Lost 0.1461 Accuracy: 0.1934
accuracy increased (0.103098 --> 0.193376).  Saving model ...
Epoch: 1/30...  Loss: 3.7919 Validation Lost 0.1417 Accuracy: 0.2185
accuracy increased (0.193376 --> 0.218483).  Saving model ...
Epoch: 1/30...  Loss: 3.4379 Validation Lost 0.1127 Accuracy: 0.2433
accuracy increased (0.218483 --> 0.243323).  Saving model ...
Epoch: 1/30...  Loss: 3.4255 Validation Lost 0.0942 Accuracy: 0.3771
accuracy increased (0.243323 --> 0.377137).  Saving model ...


KeyboardInterrupt: ignored

In [10]:
model.load_state_dict(torch.load('/content/chk.pt'))
model.eval()



vlost = 0
accuracy=0
for ii, (inputs2,labels2) in enumerate(testloader):
                optimizer.zero_grad()
                
                inputs2, labels2 = inputs2.to('cuda:0') , labels2.to('cuda:0')
                model.to('cuda:0')
                with torch.no_grad():    
                    outputs = model.forward(inputs2)
                    vlost = criterion(outputs,labels2)
                    ps = torch.exp(outputs).data
                    equality = (labels2.data == ps.max(1)[1])
                    accuracy += equality.type_as(torch.FloatTensor()).mean()
                    
vlost = vlost / len(testloader)
accuracy = accuracy /len(testloader)
            
                    
            
print("Validation Lost {:.4f}".format(vlost),
                   "Accuracy: {:.4f}".format(accuracy))
            
 

Validation Lost 0.1184 Accuracy: 0.3391


In [0]:
test = "/content/drive/My Drive/data_google"
test_dat = datasets.ImageFolder(test, transform=test_transforms)
tloader = torch.utils.data.DataLoader(test_dat, batch_size =32,shuffle = True)



In [12]:

path = F"/content/drive/My Drive/checkpoint.pt"



model.load_state_dict(torch.load('/content/chk.pt'))
model.eval()
print(PIL.PILLOW_VERSION)

tlost = 0
acuracy=0
for ii, (inputs2,labels2) in enumerate(tloader):
                optimizer.zero_grad()
                
                inputs2, labels2 = inputs2.to('cuda:0') , labels2.to('cuda:0')
                model.to('cuda:0')
                with torch.no_grad():    
                    outputs = model.forward(inputs2)
                    vlost = criterion(outputs,labels2)
                    ps = torch.exp(outputs).data
                    equality = (labels2.data == ps.max(1)[1])
                    acuracy += equality.type_as(torch.FloatTensor()).mean()
                    
tlost = tlost / len(tloader)
acuracy = acuracy /len(tloader)
            
                    
            
print("Validation Lost {:.4f}".format(tlost),
                   "Accuracy: {:.4f}".format(acuracy))
           

5.4.1
Validation Lost 0.0000 Accuracy: 0.1436
